In [59]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import wilcoxon, shapiro, ranksums
from jmetal.core.algorithm import Algorithm
from jmetal.core.quality_indicator import *
from jmetal.util.solution import print_function_values_to_file, print_variables_to_file, read_solutions

In [7]:
def generate_summary_from_experiment(input_dir, quality_indicators,
                                     reference_fronts = ''):
    reference_change = 2500
    
    if not quality_indicators:
        quality_indicators = []

    with open("QualityIndicatorSummary.csv", "w+") as of:
        of.write("Algorithm,Problem,ExecutionId,IndicatorName,IndicatorValue\n")

    for dirname, _, filenames in os.walk(input_dir):
        for filename in filenames:
            try:
                # Linux filesystem
                algorithm, problem = dirname.split("/")[-2:]
            except ValueError:
                # Windows filesystem
                algorithm, problem = dirname.split("\\")[-2:]

            if "FUN" in filename:
                solutions = read_solutions(os.path.join(dirname, filename))
                run_tag = [s for s in filename.split(".") if s.isdigit()].pop()
                for indicator in quality_indicators:
                    reference_front_file = os.path.join(reference_fronts, problem + ".pf")

                    # Add reference front if any
                    if hasattr(indicator, "reference_front"):
                        if Path(reference_front_file).is_file():
                            reference_front = []
                            with open(reference_front_file) as file:
                                for line in file:
                                    reference_front.append([float(x) for x in line.split()])

                            indicator.reference_front = reference_front
                        else:
                            logger.warning("Reference front not found at", reference_front_file)

                    result = indicator.compute([solutions[i].objectives for i in range(len(solutions))])

                    # Save quality indicator value to file
                    with open("QualityIndicatorSummary.csv", "a+") as of:
                        of.write(",".join([algorithm, problem, run_tag, indicator.get_short_name(), str(result)]))
                        of.write("\n")

In [8]:
generate_summary_from_experiment("data-100pop-100000evals-10runs-", [InvertedGenerationalDistance(None)],
                                "resources/reference_front")

In [9]:
df = pd.read_csv("QualityIndicatorSummary.csv")

In [15]:
algorithms = df["Algorithm"].unique()
problems = df["Problem"].unique()

In [16]:
for algorithm in algorithms:
    for problem in problems:
        

array(['ZDT2', 'ZDT3', 'ZDT4', 'ZDT1', 'ZDT6'], dtype=object)

In [17]:
for algorithm in algorithms:
    df[(df["Algorithm"] == algorithm)]

In [61]:
smpso = df[(df["Algorithm"] == "SMPSO") & (df["Problem"] == "ZDT6")]["IndicatorValue"]

In [62]:
smpso_epi = df[(df["Algorithm"] == "SMPSO-epi") & (df["Problem"] == "ZDT6")]["IndicatorValue"]

In [44]:
print(wilcoxon(smpso, smpso_epi, alternative="less"))

WilcoxonResult(statistic=0.0, pvalue=0.002531016063133932)


In [71]:
for problem in problems:
    smpso = df[(df["Algorithm"] == "SMPSO") & (df["Problem"] == problem)]["IndicatorValue"]
    smpso_epi = df[(df["Algorithm"] == "SMPSO-epi") & (df["Problem"] == problem)]["IndicatorValue"]
    
    print("Problem: {}".format(problem))
    print(smpso)
    print(smpso_epi)
    print(ranksums(smpso, smpso_epi))

Problem: ZDT2
50    0.003789
51    0.003821
52    0.003756
53    0.003787
54    0.003749
55    0.003780
56    0.003791
57    0.003735
58    0.003774
59    0.003808
Name: IndicatorValue, dtype: float64
0    0.003845
1    0.003900
2    0.003898
3    0.003843
4    0.003920
5    0.003921
6    0.003911
7    0.003948
8    0.003948
9    0.003918
Name: IndicatorValue, dtype: float64
RanksumsResult(statistic=-3.779644730092272, pvalue=0.00015705228423075119)
Problem: ZDT3
60    0.004236
61    0.004254
62    0.004153
63    0.004347
64    0.004363
65    0.004250
66    0.004227
67    0.004252
68    0.004371
69    0.004381
Name: IndicatorValue, dtype: float64
10    0.004890
11    0.004776
12    0.004850
13    0.004944
14    0.004995
15    0.004941
16    0.004740
17    0.004663
18    0.004752
19    0.005105
Name: IndicatorValue, dtype: float64
RanksumsResult(statistic=-3.779644730092272, pvalue=0.00015705228423075119)
Problem: ZDT4
70    0.003677
71    0.003677
72    0.003682
73    0.003675
74    0.